In [5]:
import pandas as pd

In [6]:
df_hh = pd.read_pickle("./data/Feature Selection aggregated household load.pkl")
df_hp = pd.read_pickle("./data/Feature Selection aggregated heat pump load.pkl")
df_comb = pd.read_pickle("./data/Feature Selection aggregated energy community load.pkl")

In [7]:
target_dfs = [df_hh,df_hp,df_comb]
df_hh

,RELATIVE_HUMIDITY:TOTAL,hour_cos,WIND_GUST_SPEED:TOTAL,APPARENT_TEMPERATURE:TOTAL ROLLING 48,APPARENT_TEMPERATURE:TOTAL ROLLING 24,load_same_hour_last_7_days,hour_sin,Agg Load
date,,,,,,,,
2019-01-02 23:00:00+00:00,95.0,1.000000,4.652778,-2.167969,-3.426389,703.202163,-2.449294e-16,5071.464472
2019-01-03 00:00:00+00:00,95.0,1.000000,4.953704,-2.234288,-3.402778,1457.902353,0.000000e+00,3909.119887
2019-01-03 01:00:00+00:00,95.0,0.962917,5.532407,-2.303385,-3.397917,1334.682895,2.697968e-01,4004.711000
2019-01-03 02:00:00+00:00,95.0,0.854419,6.087963,-2.384201,-3.395139,1274.462116,5.195840e-01,3886.548474
2019-01-03 03:00:00+00:00,95.0,0.682553,6.342593,-2.475694,-3.372569,1235.234996,7.308360e-01,3808.471335
...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00+00:00,90.0,0.460065,4.444445,-0.322743,-0.266319,10903.147091,-8.878852e-01,10689.887782
2020-12-31 20:00:00+00:00,90.0,0.682553,4.444445,-0.322743,-0.266319,9053.342728,-7.308360e-01,8392.824193
2020-12-31 21:00:00+00:00,90.0,0.854419,4.444445,-0.322743,-0.266319,7642.159802,-5.195840e-01,9028.434408


In [8]:
# building day before data columns
deltas = [24,49]

for df in target_dfs: 
    for delt in range(deltas[0],deltas[1]):
        df["Agg Load - "+str(delt)] = df["Agg Load"].shift(delt)
    df.dropna(inplace=True)


In [14]:
def get_shifted_value(row, delt_hours):
    target_time = row['first_of_day'] - pd.Timedelta(hours=delt_hours)
    value = df.loc[df['date'] == target_time, 'Agg Load']
    return value.iloc[0] if not value.empty else None

deltas = [24,49mome]
for df in target_dfs:
    df.reset_index(inplace=True)
    df['first_of_day'] = df.groupby(df['date'].dt.date)['date'].transform('min')

    for delt in range(deltas[0], deltas[1]): # mapping delta values
        df[f"Agg Load - {delt}"] = df.apply(get_shifted_value, delt_hours=delt, axis=1)

    df.drop(columns=['first_of_day'], inplace=True)
    
    df.dropna(inplace=True)

In [15]:
for df in target_dfs:
    df.set_index("date", inplace=True)

In [16]:
first_timesteps = df_hh.resample('MS').first()

# Convert the index back to a list
first_timesteps_list = first_timesteps.index.tolist()[-12:]

In [17]:
first_timesteps_list

[Timestamp('2020-01-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-02-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-03-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-04-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-05-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-06-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-07-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-08-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-09-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-10-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-11-01 00:00:00+0000', tz='UTC', freq='MS'),
 Timestamp('2020-12-01 00:00:00+0000', tz='UTC', freq='MS')]

In [18]:
target_dfs_strs = ["HH","HP","Comb"]

for idx, target_df in enumerate(target_dfs):
    for date in first_timesteps_list:
        start_date = date - pd.DateOffset(months=12)
        end_date = date - pd.DateOffset(days=1)
        data_range = target_df[start_date:end_date]
        
        month = str(date.month)
        data_range.to_pickle("./data/traintest/2020 "+target_dfs_strs[idx]+" "+month+" train.pkl")
        
        end_date_test = date  + pd.DateOffset(months=1)
        
        if date.month < 12:
            data_range = target_df[date:end_date_test][:-1]
        else:
            data_range = target_df[date:end_date_test]
        data_range.to_pickle("./data/traintest/2020 "+target_dfs_strs[idx]+" "+month+" test.pkl")


In [21]:
data_range

,RELATIVE_HUMIDITY:TOTAL,hour_cos,WIND_GUST_SPEED:TOTAL,APPARENT_TEMPERATURE:TOTAL ROLLING 48,TEMPERATURE:TOTAL,APPARENT_TEMPERATURE:TOTAL,APPARENT_TEMPERATURE:TOTAL ROLLING 24,load_same_hour_last_7_days,hour_sin,Agg Load,...,Agg Load - 39,Agg Load - 40,Agg Load - 41,Agg Load - 42,Agg Load - 43,Agg Load - 44,Agg Load - 45,Agg Load - 46,Agg Load - 47,Agg Load - 48
date,,,,,,,,,,,,,,,,,,,,,
2020-12-01 00:00:00+00:00,93.750000,1.000000,3.333333,-0.468576,3.516667,3.300000,1.021181,17042.889567,0.000000e+00,23911.300093,...,32461.023247,32875.312677,28908.493315,27811.409315,25843.996163,24470.926467,21987.464946,22585.755369,21114.050813,22204.815620
2020-12-01 01:00:00+00:00,95.000000,0.962917,3.819444,-0.453472,3.275000,1.566667,1.163889,16760.601245,2.697968e-01,17434.825100,...,32461.023247,32875.312677,28908.493315,27811.409315,25843.996163,24470.926467,21987.464946,22585.755369,21114.050813,22204.815620
2020-12-01 02:00:00+00:00,93.333333,0.854419,3.888889,-0.441319,3.150000,1.441667,1.283681,17263.290237,5.195840e-01,20289.377180,...,32461.023247,32875.312677,28908.493315,27811.409315,25843.996163,24470.926467,21987.464946,22585.755369,21114.050813,22204.815620
2020-12-01 03:00:00+00:00,95.000000,0.682553,3.888889,-0.423958,3.025000,1.658333,1.398264,18601.533447,7.308360e-01,19735.208932,...,32461.023247,32875.312677,28908.493315,27811.409315,25843.996163,24470.926467,21987.464946,22585.755369,21114.050813,22204.815620
2020-12-01 04:00:00+00:00,93.333333,0.460065,3.888889,-0.349479,3.091667,2.791667,1.561806,19055.454398,8.878852e-01,22814.185894,...,32461.023247,32875.312677,28908.493315,27811.409315,25843.996163,24470.926467,21987.464946,22585.755369,21114.050813,22204.815620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00+00:00,90.000000,0.460065,4.444445,-0.322743,1.200000,1.200000,-0.266319,27326.356794,-8.878852e-01,30587.419588,...,32006.954652,34100.498944,29346.754931,27164.083498,23913.964554,23422.931836,22073.092529,19917.219745,17116.549122,17673.820232
2020-12-31 20:00:00+00:00,90.000000,0.682553,4.444445,-0.322743,1.200000,1.200000,-0.266319,24185.633367,-7.308360e-01,24249.875910,...,32006.954652,34100.498944,29346.754931,27164.083498,23913.964554,23422.931836,22073.092529,19917.219745,17116.549122,17673.820232
2020-12-31 21:00:00+00:00,90.000000,0.854419,4.444445,-0.322743,1.200000,1.200000,-0.266319,21659.159769,-5.195840e-01,24439.704660,...,32006.954652,34100.498944,29346.754931,27164.083498,23913.964554,23422.931836,22073.092529,19917.219745,17116.549122,17673.820232


In [22]:

target_dfs_strs = ["HH","HP","Comb"]
for idx, target_df in enumerate(target_dfs):
    start_date = '2019-01-01'
    end_date = '2019-06-30'

    filtered_df = target_df[(target_df.index >= start_date) & (target_df.index <= end_date)]
    filtered_df.to_pickle("./data/hyperparameter train "+target_dfs_strs[idx]+".pkl")
    
    start_date = '2019-07-01'
    end_date = '2019-12-31'

    filtered_df = target_df[(target_df.index >= start_date) & (target_df.index <= end_date)]
    filtered_df = filtered_df[filtered_df.index.month.isin([7, 9, 11])]
    filtered_df.to_pickle("./data/hyperparameter test "+target_dfs_strs[idx]+".pkl")


In [23]:
filtered_df

,RELATIVE_HUMIDITY:TOTAL,hour_cos,WIND_GUST_SPEED:TOTAL,APPARENT_TEMPERATURE:TOTAL ROLLING 48,TEMPERATURE:TOTAL,APPARENT_TEMPERATURE:TOTAL,APPARENT_TEMPERATURE:TOTAL ROLLING 24,load_same_hour_last_7_days,hour_sin,Agg Load,...,Agg Load - 39,Agg Load - 40,Agg Load - 41,Agg Load - 42,Agg Load - 43,Agg Load - 44,Agg Load - 45,Agg Load - 46,Agg Load - 47,Agg Load - 48
date,,,,,,,,,,,,,,,,,,,,,
2019-07-01 00:00:00+00:00,68.750000,1.000000,7.893518,23.841667,14.400000,14.400000,20.103472,5298.899066,0.000000e+00,4962.378030,...,13409.323259,11559.998122,10293.337673,8463.418024,6206.104726,5072.865080,4868.288137,4942.328501,5269.991804,5077.420280
2019-07-01 01:00:00+00:00,70.000000,0.962917,7.453704,23.807292,13.458333,13.458333,19.980555,5238.866643,2.697968e-01,5042.450124,...,13409.323259,11559.998122,10293.337673,8463.418024,6206.104726,5072.865080,4868.288137,4942.328501,5269.991804,5077.420280
2019-07-01 02:00:00+00:00,70.833333,0.854419,7.222222,23.758680,12.641666,12.641666,19.863194,5110.197826,5.195840e-01,5117.091389,...,13409.323259,11559.998122,10293.337673,8463.418024,6206.104726,5072.865080,4868.288137,4942.328501,5269.991804,5077.420280
2019-07-01 03:00:00+00:00,74.166667,0.682553,7.337963,23.715625,12.858333,12.858333,19.770486,5789.725060,7.308360e-01,6890.256693,...,13409.323259,11559.998122,10293.337673,8463.418024,6206.104726,5072.865080,4868.288137,4942.328501,5269.991804,5077.420280
2019-07-01 04:00:00+00:00,70.000000,0.460065,8.148148,23.677430,13.025000,13.025000,19.673958,6015.497258,8.878852e-01,6927.477996,...,13409.323259,11559.998122,10293.337673,8463.418024,6206.104726,5072.865080,4868.288137,4942.328501,5269.991804,5077.420280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-30 19:00:00+00:00,95.000000,0.460065,2.407407,-0.215278,-0.700000,-0.700000,-1.480903,29541.660989,-8.878852e-01,38903.311978,...,20986.892830,28371.661320,23440.074210,27690.707737,23985.823600,18217.421845,19911.730038,16919.894534,15386.491489,21443.691061
2019-11-30 20:00:00+00:00,95.000000,0.682553,2.222222,-0.281944,-1.050000,-1.050000,-1.466319,25866.128568,-7.308360e-01,43043.725286,...,20986.892830,28371.661320,23440.074210,27690.707737,23985.823600,18217.421845,19911.730038,16919.894534,15386.491489,21443.691061
2019-11-30 21:00:00+00:00,95.000000,0.854419,2.291667,-0.347396,-0.941667,-0.941667,-1.435417,25201.467664,-5.195840e-01,32924.968188,...,20986.892830,28371.661320,23440.074210,27690.707737,23985.823600,18217.421845,19911.730038,16919.894534,15386.491489,21443.691061
